<a href="https://colab.research.google.com/github/Manos-Mak120/-/blob/Articles-Scrapper/TRUMP_ARTICLES_ATHENS_VOICE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_article_elements_from_soup(soup, article_url):
  # απομόνωση του block με το περιεχόμενο του άρθρου από όλη τη σούπα της σελίδας
  article_block = soup.find("div", {"class": "articleSingle articleSingle--default"})
  # article url
  article_url = article_url
  # title
  try:
    title = article_block.find("h1", {"class": "articleSingle__title"}).text.strip()
  except:
    # try:
    #   title = soup.find("div", {"class": "container is-fullhd p-3"}).h1.text.strip() # προσθήκη για να πιάνει και τον τίτλο όταν βρίσκεται σε αυτό το div
    # except:
    title=""
  # datetime
  try:
    datetime_str = article_block.find("time")["datetime"]
  except:
    try:
      datetime_str = soup.find("li", {"class": "actionsList__item actionsList__item--date"}).find("time")["datetime"]
    except:
      datetime_str = ""
      date_str = ""
      time_str = ""
  if datetime_str != "":
    date_str = datetime_str.split("T")[0]
    time_str = datetime_str.split("T")[1].split("Z")[0]
  # section
  try:
    section = article_block.find("a", class_=lambda c: c and "articleSingle__category" in c).text.strip()
  except:
    section = ""
  # author
  try:
    author = article_block.find("a", class_=lambda c: c and "actionsList__link" in c).find("span").text.strip()
  except:
    # try:
    #   author = soup.find("div", {"class": "container is-fullhd p-3"}).find("a", {"rel": "author"}).text.strip() # προσθήκη
    # except:
    author = ""
  #subtitle
  try:
    subtitle = article_block.find("div", {"class": "articleSingle__intro"}).text.strip()
  except:
    subtitle = ""
  # excerpt
  try:
    excerpt = article_block.find("div", {"class": "articleSingle__summaryContent"}).find("p").text.strip()
  except:
    # try:
    #   excerpt = soup.find("div", {"class": "container is-fullhd p-3"}).find("div", {"class": "newsletter-description nx-excerpt my-5"}).text.strip() # προσθήκη
    # except:
    excerpt=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", class_="articleSingle__description").find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
    text_body = " ".join(text_body_textsL)
  except:
    text_body=""
  # feature image
  try:
    feat_img_src = article_block.find("div", class_="articleSingle__image").find('img')["src"]
    feat_img_link = "https://www.athensvoice.gr"+feat_img_src
  except:
    feat_img_link=""


  # Δημιουργία λεξικού με τα data του άρθρου
  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excerpt": excerpt, "subtitle": subtitle, "text_body": text_body, "feat_img_link": feat_img_link}

  return articleD

In [ ]:
def scrape_athensvoice_articles_pages(teasers_df):
  all_articles_dataL = []
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):

    article_url = row['article_url']
    soup = get_webpage_soup(article_url)
    if soup is not None:
      articleD = get_article_elements_from_soup(soup, article_url)
      all_articles_dataL.append(articleD)
    else:
      # Optional: Print or log the URL that failed to scrape
      print(f"Warning: Could not scrape {article_url}")
    # articleD = get_article_elements_from_soup(soup, article_url)
    # all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetched {len(df)} articles.")
  return df

**Δοκιμή του scraper**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trump_df = pd.read_csv("/content/drive/MyDrive/Athens.voice/Scrapper/_trump_athens_voices_df.csv")

In [ ]:
articles_df = scrape_athensvoice_articles_pages(trump_df)
articles_df

Processing rows: 100%|██████████| 1000/1000 [1:01:19<00:00,  3.68s/it]


Scraping completed. Fetched 1000 articles.


,article_url,title,date,time,section,author,excerpt,subtitle,text_body,feat_img_link
0,https://www.athensvoice.gr/epikairotita/diethn...,Ντόναλντ Τραμπ και Έλον Μασκ: Οι 10 ημέρες που...,2025-06-07,06:01:52,Κοσμος,Newsroom,ΗΠΑ: Πώς η σχέση Τραμπ - Μασκ κατέρρευσε σε 10...,"Το χρονικό της σύγκρουσης - Η δημόσια ρήξη, οι...","Η σύμπραξη του Ντόναλντ Τραμπ, ηγέτη της πλουσ...",https://www.athensvoice.gr/images/1074x600/jpg...
1,https://www.athensvoice.gr/epikairotita/diethn...,Ο Τραμπ προσέφυγε στο Ανώτατο Δικαστήριο για ν...,2025-06-06,16:48:00,Κοσμος,Newsroom,Το κλείσιμο του υπουργείου Παιδείας ήταν εδώ κ...,Η εκπαιδευτική πολιτική θα περάσει εξολοκλήρου...,Η κυβέρνηση του προέδρου Ντόναλντ Τραμπ ζήτησε...,https://www.athensvoice.gr/images/1074x600/1/j...
2,https://www.athensvoice.gr/epikairotita/diethn...,Tesla: Κατάρρευση μετοχών εν μέσω της διαμάχης...,2025-06-06,13:41:00,Κοσμος,Newsroom,Η μετοχή της Tesla κατέρρευσε λόγω της διαμάχη...,Δραματική κλιμάκωση της έντασης μεταξύ δύο από...,Οι μετοχές της Tesla κατέγραψαν πτώση περίπου ...,https://www.athensvoice.gr/images/1074x600/jpg...
3,https://www.athensvoice.gr/epikairotita/diethn...,Απόλυτο διαζύγιο: Η θεαματική κατάρρευση της σ...,2025-06-06,09:19:54,Κοσμος,Newsroom,"Η συμμαχία Τραμπ-Μασκ διαλύεται θεαματικά, φέρ...",Μια σχέση εξουσίας και συμφερόντων μετατρέπετα...,Όταν ο Ντόναλντ Τραμπ συναντήθηκε ιδιωτικά προ...,https://www.athensvoice.gr/images/1074x600/jpg...
4,https://www.athensvoice.gr/epikairotita/diethn...,Ο Τραμπ είχε ταξιδέψει τουλάχιστον 7 φορές με ...,2025-06-06,06:57:33,Κοσμος,Newsroom,Ο Τραμπ είχε ταξιδέψει τουλάχιστον 7 φορές με ...,Εκεί όπου ο χρηματιστής επιδιδόταν σε σεξουαλι...,Με μια ανάρτηση στο X που συνδέει τον Ντόναλντ...,https://www.athensvoice.gr/images/1074x600/jpg...
...,...,...,...,...,...,...,...,...,...,...
995,https://www.athensvoice.gr/epikairotita/diethn...,Ρόμπερτ Κένεντι Τζούνιορ: Υποψήφιος υπουργός Υ...,2024-11-14,21:11:00,Κοσμος,Newsroom,Ρόμπερτ Κένεντι Τζούνιορ: Φαβορί για υπουργός ...,Είναι ως γνωστόν οπαδός θεωριών συνομωσίας και...,Ο εκλεγμένος πρόεδρος των ΗΠΑ Ντόναλντ Τραμπ α...,https://www.athensvoice.gr/images/1074x600/1/j...
996,https://www.athensvoice.gr/viral/trending-now/...,Τραμπ και Μπάιντεν φίλοι για πάντα μέσω AI - Τ...,2024-11-14,16:22:05,Trending Now,Newsroom,Τραμπ και Μπάιντεν - Το viral βίντεο που δημιο...,Την αρχή μιας... υπέροχης φιλίας ανέλαβε να σχ...,Τον γύρο του διαδικτύου κάνει ένα βίντεο που δ...,https://www.athensvoice.gr/images/1074x600/jpg...
997,https://www.athensvoice.gr/epikairotita/diethn...,Mατ Γκετζ: Ο υπουργός Δικαιοσύνης που έχει ερε...,2024-11-14,15:24:43,Κοσμος,Newsroom,Ματ Γκετζ: Ποιος είναι ο αμφιλεγόμενος υπουργό...,Aντιδράσεις ακόμα και τους Ρεπουμπλικάνους,Ο εκλεγμένος πρόεδρος Ντόναλντ Τραμπ όρισε την...,https://www.athensvoice.gr/images/1074x600/1/j...
998,https://www.athensvoice.gr/epikairotita/diethn...,Δήμαρχος Λονδίνου: Ο Τραμπ με στοχοποίησε λόγω...,2024-11-14,15:07:29,Κοσμος,Newsroom,Ο δήμαρχος του Λονδίνου Σαντίκ Καν κατηγορεί τ...,Η σχέση μεταξύ των δύο ανδρών ήταν ανέκαθεν τε...,Eναντίον του Ντόναλντ Τραμπ στρέφεται εκ νέου ...,https://www.athensvoice.gr/images/1074x600/jpg...
